In [1]:
import pandas as pd

In [2]:
import re

In [146]:
in_office_ = pd.read_csv("IT_Project_Manager_wages.csv")
remote_wages = pd.read_csv("IT_Project_Manager_wages_remote.csv")
office_space = pd.read_csv("Office_rent.csv")  
remote_expenses = pd.read_csv("remoteExpenses.csv")
commute_costs = pd.read_csv("Commute Cost.csv")
electricity_rates = pd.read_csv("electricity_rates.csv")
col = pd.read_csv("cost_of_living_index.csv")

### Cleaning Wages datasets ###

In [149]:
# Function to clean and extract the salary range

def clean_salary(salary_str):
    if isinstance(salary_str, str) and salary_str != 'N/A':
        # Remove any non-numeric characters except for numbers, hyphen, period, and commas
        salary_str = re.sub(r'[^0-9\.\-,]', '', salary_str)  # Remove any unwanted characters
        salary_str = salary_str.replace(",", "")  # Remove commas if present

        # If there is a range, split and calculate the average salary
        salary_parts = salary_str.split('-')

        if len(salary_parts) == 2:
            # Remove any trailing characters like periods and calculate the average salary
            salary_parts = [s.strip(' .') for s in salary_parts]  # Clean up any trailing spaces/periods
            avg_salary = (float(salary_parts[0]) + float(salary_parts[1])) / 2
        else:
            salary_str = salary_str.strip(' .')  # Clean up the salary string (remove any trailing periods or spaces)
            avg_salary = float(salary_str) if salary_str else None

        return avg_salary * 1000  # Convert to full salary amount (e.g., $100K becomes 100000)
    else:
        return None

# Clean the salary columns
in_office_wages['Salary'] = in_office_wages['Salary'].apply(clean_salary)
remote_wages['Salary'] = remote_wages['Salary'].apply(clean_salary)

In [6]:
# Standardize the Location format
def standardize_location(location):
    parts = location.split(', ')
    if len(parts) == 2:
        city, state = parts[0], parts[1]
    else:
        city, state = None, parts[0]  # State-only entry
    return city, state

# Apply the location standardization
in_office_wages[['City', 'State']] = in_office_wages['Location'].apply(standardize_location).apply(pd.Series)
remote_wages[['City', 'State']] = remote_wages['Location'].apply(standardize_location).apply(pd.Series)


In [151]:
in_office_wages.head()

,Keyword,Company,Job_title,City,State,Unnamed: 5,Salary,Unnamed: 7,Unnamed: 8
0,software developer,"CyberBalance, LLC",Senior .NET Developers to Modernize Cybersecur...,Remote,NaN,NaN,NaN,NaN,NaN
1,software developer,eComSystems,Full Stack Website Developer,Sarasota,FL,NaN,120000.0,NaN,NaN
2,software developer,"Bend-Tech, LLC",Entry Level Applications Engineer (Background ...,Osceola,WI,NaN,73500.0,NaN,NaN
3,software developer,Ceresti Health,Senior Golang Backend Developer,Norwell,MA,NaN,135000.0,NaN,NaN
4,software developer,"CyberBalance, LLC",Senior .NET Developer – UI/UX-Focused Cybersec...,Remote,NaN,NaN,NaN,NaN,NaN


In [153]:
remote_wages.head()

,Keyword,Company,Job_title,City,State,Salary
0,software developer,"CyberBalance, LLC",Senior .NET Developers to Modernize Cybersecur...,Remote,NaN,NaN
1,software developer,eComSystems,Full Stack Website Developer,Sarasota,FL,120000.0
2,software developer,"Bend-Tech, LLC",Entry Level Applications Engineer (Background ...,Osceola,WI,73500.0
3,software developer,Ceresti Health,Senior Golang Backend Developer,Norwell,MA,135000.0
4,software developer,"CyberBalance, LLC",Senior .NET Developer – UI/UX-Focused Cybersec...,Remote,NaN,NaN


In [155]:
in_office_wages.shape

(239, 9)

In [157]:
remote_wages.shape

(537, 6)

In [163]:
# Drop rows where state is 'remote'
remote_wages = remote_wages[~remote_wages[['City', 'State']].apply(lambda x: x.str.contains('remote', case=False, na=False)).any(axis=1)]

In [165]:
# Merge the datasets on City and State
wages = pd.merge(in_office_wages, remote_wages, on=['City', 'State'], how='left', suffixes=('_in_office', '_remote'))

In [171]:
# Display merged dataset
wages.head()

,Keyword_in_office,Company_in_office,Job_title_in_office,City,State,Unnamed: 5,Salary_in_office,Unnamed: 7,Unnamed: 8,Keyword_remote,Company_remote,Job_title_remote,Salary_remote
1,software developer,eComSystems,Full Stack Website Developer,Sarasota,FL,NaN,120000.0,NaN,NaN,software developer,eComSystems,Full Stack Website Developer,120000.0
2,software developer,"Bend-Tech, LLC",Entry Level Applications Engineer (Background ...,Osceola,WI,NaN,73500.0,NaN,NaN,software developer,"Bend-Tech, LLC",Entry Level Applications Engineer (Background ...,73500.0
3,software developer,Ceresti Health,Senior Golang Backend Developer,Norwell,MA,NaN,135000.0,NaN,NaN,software developer,Ceresti Health,Senior Golang Backend Developer,135000.0
5,software developer,Futurex,Senior Front End Web Developer,Bulverde,TX,NaN,82000.0,NaN,NaN,software developer,Futurex,Senior Front End Web Developer,82000.0
6,software developer,Futurex,Senior Front End Web Developer,Bulverde,TX,NaN,82000.0,NaN,NaN,software developer,Futurex,Senior Software Engineer,121500.0


In [173]:
wages.shape

(1625, 13)

In [15]:
wages.isna().sum()

Role_in_office          0
Company_in_office       0
Job_title               0
Location_in_office      0
Salary_in_office      159
Experience              0
City                  169
State                   0
Role_remote           627
Title                 794
Company_remote        878
Location_remote       627
Salary_remote         634
dtype: int64

In [16]:
# Drop unneccessary columns
wages = wages.drop(['Location_in_office','Role_remote','Title','Company_remote','Location_remote'], axis=1)

In [17]:
wages['State'].unique()

array(['GA', 'TX', 'FL', 'OH', 'CA', 'NY', 'United States', 'DC', 'MI',
       'MA', 'VA', 'AR', 'WI', 'NH', 'NC', 'IL', 'UT', 'MN', 'NJ',
       'Virginia', 'Georgia', 'WV', 'SC', 'WA', 'Westcliffe',
       'North Carolina', 'AL', 'IA', 'KY', 'IN', 'KS', 'PA', 'MD', 'MS',
       'OK', 'TN', 'Remote', 'LA', 'ND', 'NM', 'Indiana', 'Florida', 'OR',
       'AZ', 'MO', 'AK', 'Ohio', 'Alabama', 'CO', 'Idaho', 'DE',
       'South Carolina', 'CT', 'NE', 'RI', 'Arizona', 'Texas', 'NV',
       'California', 'Missouri', 'Minneapolis-Saint Paul',
       'Colonial Park', 'Connecticut', 'Brighton', 'New Jersey', 'VT',
       'HI', 'PR', 'Long Island-Queens', 'Brookhaven', 'Manhattan'],
      dtype=object)

In [18]:
# Drop rows where the 'State' column is either 'United States' or 'Remote'
wages = wages[~wages['State'].isin(['United States', 'Remote'])]
wages.shape

(750, 8)

In [19]:
# Dictionary mapping state abbreviations to full state names
state_abbreviation_map = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC':'District of Columbia', 'Unknown':'Unknown'
}

# Remove leading/trailing spaces and convert to uppercase
wages['State'] = wages['State'].str.strip().str.upper()

# Fill NaN values with 'Unknown'
wages['State'].fillna('Unknown', inplace=True)

# Now apply the mapping
wages['State'] = wages['State'].map(state_abbreviation_map)
office_space['State'] = office_space['State'].map(state_abbreviation_map)
col['State'] = col['State'].map(state_abbreviation_map)
# Display a sample of the result
print(wages[['State']].head())
print(office_space['State'].head())
print(col['State'].head())

     State
0  Georgia
1    Texas
2  Florida
3     Ohio
4  Florida
0    Alabama
1    Alabama
2    Alabama
3    Alabama
4    Alabama
Name: State, dtype: object
0         Texas
1      Michigan
2          Ohio
3    New Mexico
4       Georgia
Name: State, dtype: object


/var/folders/sx/60_bj5px0wx1bnj278_ft4b40000gn/T/ipykernel_40180/4151751402.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wages['State'].fillna('Unknown', inplace=True)


In [20]:
# Check for Washington D.C. and update the state
wages['State'] = wages.apply(
    lambda row: 'DC' if row['City'] == 'Washington' and pd.isna(row['State']) else row['State'],
    axis=1
)

In [21]:
wages['City'].nunique

<bound method IndexOpsMixin.nunique of 0          Alpharetta
1      Farmers Branch
2              Naples
3           Cleveland
4           Opa-locka
            ...      
873           Atlanta
874              None
875          Brooklyn
876           Herndon
877        Washington
Name: City, Length: 750, dtype: object>

### Merging the datasets ###

In [23]:
# Extracting only required columns
wages = wages[["Role_in_office","Job_title", "Company_in_office", "City", "State", "Experience", "Salary_in_office", "Salary_remote"]]

office_space = office_space[["City", "State", "Avg. Rent"]]

remote_expenses = remote_expenses[["State",
                                   "Average monthly payment","Taxes, Fees and Government Surcharges on Wireless Service (%)"]]

commute_costs = commute_costs[["City", "State", "Annual Commute Cost"]]

In [24]:
# Rename columns
wages.rename(columns={"Role_in_office": "Role"}, inplace=True)
office_space.rename(columns={"Avg. Rent":"Avg.OfficeRent_per_sqft"}, inplace=True)
remote_expenses.rename(columns={"Average monthly payment": "Avg internet monthly payment"}, inplace=True)

In [25]:
# Merge wages with office space costs on 'State'
df = wages.merge(office_space, on=["City","State"], how="left")

In [26]:
# Merge with remote expenses on 'State'
df = df.merge(remote_expenses, on="State", how="left")

In [27]:
# Merge with commute costs on 'City' and 'State'
df = df.merge(commute_costs, on=["City", "State"], how="left")

In [28]:
# Merge with electricity on 'State'
df = df.merge(electricity_rates, on="State", how="left")

In [29]:
# Merge with COI on 'City' and 'State'
df = df.merge(col, on=["City", "State"], how="left")

In [30]:
df.tail()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,"Taxes, Fees and Government Surcharges on Wireless Service (%)",Annual Commute Cost,Residential electricity rates,Cost of Living Index
1073,IT project manager,PROJECT DIRECTOR,HRA/DEPT OF SOCIAL SERVICES,None,NaN,Senior-level,119500.0,NaN,NaN,NaN,NaN,NaN,NaN,115.9
1074,IT project manager,PROJECT DIRECTOR,HRA/DEPT OF SOCIAL SERVICES,None,NaN,Senior-level,119500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,IT project manager,"PROJECT DIRECTOR, IMAGING & ENTERPRISE DATA PR...",HRA/DEPT OF SOCIAL SERVICES,Brooklyn,New York,Senior-level,119500.0,NaN,$44.85,NaN,NaN,NaN,24.37 ¢/kWh,NaN
1076,IT project manager,Senior Capture Director Intelligence and Homel...,General Dynamics Information Technology,Herndon,Virginia,Senior-level,250500.0,NaN,$29.67,$86.70,7.72,NaN,14.46 ¢/kWh,NaN
1077,IT project manager,WDLL Pathway Director (IT),University of the District of Columbia,Washington,DC,Senior-level,77000.0,NaN,$52.54,NaN,NaN,"$11,067.07",16.5 ¢/kWh,120.1


### Cleaning the merged dataset ###

In [32]:
df.shape

(1078, 14)

In [33]:
df.isna().sum()

Role                                                               0
Job_title                                                          0
Company_in_office                                                  0
City                                                             369
State                                                            370
Experience                                                         0
Salary_in_office                                                 205
Salary_remote                                                    768
Avg.OfficeRent_per_sqft                                          553
Avg internet monthly payment                                     824
Taxes, Fees and Government Surcharges on Wireless Service (%)    824
Annual Commute Cost                                              680
Residential electricity rates                                    370
Cost of Living Index                                             353
dtype: int64

In [34]:
# Drop duplicate rows
df = df.drop_duplicates()

# Remove rows with NaN values in 'Salary_in_office' column
df = df.dropna(subset=['Salary_in_office'])

df.shape

(801, 14)

In [35]:
# Remove '$' and convert to float
df['Avg.OfficeRent_per_sqft'] = df['Avg.OfficeRent_per_sqft'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['Avg internet monthly payment'] = df['Avg internet monthly payment'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['Annual Commute Cost'] = (df['Annual Commute Cost'].str.replace('[\$,]', '', regex=True).astype(float))
df['Electricity Price (cents per kWh)'] = df['Residential electricity rates'].replace({'¢/kWh': '', ' ': ''}, regex=True).astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
/var/folders/sx/60_bj5px0wx1bnj278_ft4b40000gn/T/ipykernel_40180/2734183873.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['Avg.OfficeRent_per_sqft'] = df['Avg.OfficeRent_per_sqft'].replace({'\$': '', ',': ''}, regex=True).astype(float)
/var/folders/sx/60_bj5px0wx1bnj278_ft4b40000gn/T/ipykernel_40180/2734183873.py:3: SyntaxWarning: invalid escape sequence '\$'
  df['Avg internet monthly payment'] = df['Avg internet monthly payment'].replace({'\$': '', ',': ''}, regex=True).astype(float)
/var/folders/sx/60_bj5px0wx1bnj278_ft4b40000gn/T/ipykernel_40180/2734183873.py:4: SyntaxWarning: invalid escape sequence '\$'
  df['Annual Commute Cost'] = (df['Annual Commute Cost'].str.replace('[\$,

In [36]:
df = df.drop(['Residential electricity rates','Taxes, Fees and Government Surcharges on Wireless Service (%)'], axis=1)

In [116]:
col.shape

(519, 3)

In [38]:
#Columns to fill NaN values
columns_to_fill = [ 
    'Avg.OfficeRent_per_sqft',
    'Electricity Price (cents per kWh)', 
    'Avg internet monthly payment', 
    'Annual Commute Cost',
    'Cost of Living Index'
]

# Fill NaN values with the mean of the corresponding state
for column in columns_to_fill:
    df[column] = df.groupby('State')[column].transform(lambda x: x.fillna(x.mean()))


# office_space['Avg.OfficeRent_per_sqft'] = office_space['Avg.OfficeRent_per_sqft'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# electricity_rates['Residential electricity rates'] = electricity_rates['Residential electricity rates'].replace({'\$': '', ',': '', '¢/kWh': ''}, regex=True).astype(float)
# remote_expenses['Avg internet monthly payment'] = remote_expenses['Avg internet monthly payment'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# commute_costs['Annual Commute Cost'] = commute_costs['Annual Commute Cost'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# col['Cost of Living Index'] = col['Cost of Living Index'].replace({'\$': '', ',': ''}, regex=True).astype(float)


# df['Avg.OfficeRent_per_sqft'] = office_space.groupby('State')['Avg.OfficeRent_per_sqft'].transform(lambda x: x.fillna(x.mean()))
# df['Electricity Price (cents per kWh)'] = electricity_rates.groupby('State')['Residential electricity rates'].transform(lambda x: x.fillna(x.mean()))
# df['Avg internet monthly payment'] = remote_expenses.groupby('State')['Avg internet monthly payment'].transform(lambda x: x.fillna(x.mean()))
# df['Annual Commute Cost'] = commute_costs.groupby('State')['Annual Commute Cost'].transform(lambda x: x.fillna(x.mean()))
# df['Cost of Living Index'] = col.groupby('State')['Cost of Living Index'].transform(lambda x: x.fillna(x.mean()))

In [39]:
# Fill NaN values with the average internet cost
df['Avg internet monthly payment'] = df['Avg internet monthly payment'].fillna(df['Avg internet monthly payment'].mean())

In [40]:
df.head()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,Annual Commute Cost,Cost of Living Index,Electricity Price (cents per kWh)
1,IT project manager,Assistant Project Manager,Yogurtland,Farmers Branch,Texas,Entry-level,78500.0,NaN,31.260000,81.659353,6119.970405,90.5,15.32
2,IT project manager,Assistant Program Manager,Azimuth Technology,Naples,Florida,Entry-level,43500.0,NaN,34.280000,83.250000,5147.828333,89.4,14.20
3,IT project manager,Junior Project Manager (IT Aviation),IDMR Solutions Inc,Cleveland,Ohio,Entry-level,44500.0,NaN,20.790000,81.659353,3936.900000,85.8,15.98
4,IT project manager,JUNIOR PROJECT MANAGER/PROJECT ANALYST M/F/D,F. LIST FLORIDA LLC,Opa-locka,Florida,Entry-level,72500.0,NaN,32.934118,83.250000,5147.828333,87.3,14.20
5,IT project manager,"Junior Project Manager in Orlando, FL & San Di...",Exponents Insta USA Inc,San Diego,California,Entry-level,55000.0,NaN,41.600000,81.659353,6210.030000,105.4,30.55


In [41]:
#Rounding up decimals
df[['Avg.OfficeRent_per_sqft','Avg internet monthly payment','Annual Commute Cost','Cost of Living Index']] = df[['Avg.OfficeRent_per_sqft','Avg internet monthly payment','Annual Commute Cost','Cost of Living Index']].round(2)

In [42]:
df.head()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,Annual Commute Cost,Cost of Living Index,Electricity Price (cents per kWh)
1,IT project manager,Assistant Project Manager,Yogurtland,Farmers Branch,Texas,Entry-level,78500.0,NaN,31.26,81.66,6119.97,90.5,15.32
2,IT project manager,Assistant Program Manager,Azimuth Technology,Naples,Florida,Entry-level,43500.0,NaN,34.28,83.25,5147.83,89.4,14.20
3,IT project manager,Junior Project Manager (IT Aviation),IDMR Solutions Inc,Cleveland,Ohio,Entry-level,44500.0,NaN,20.79,81.66,3936.90,85.8,15.98
4,IT project manager,JUNIOR PROJECT MANAGER/PROJECT ANALYST M/F/D,F. LIST FLORIDA LLC,Opa-locka,Florida,Entry-level,72500.0,NaN,32.93,83.25,5147.83,87.3,14.20
5,IT project manager,"Junior Project Manager in Orlando, FL & San Di...",Exponents Insta USA Inc,San Diego,California,Entry-level,55000.0,NaN,41.60,81.66,6210.03,105.4,30.55


### Feature Engineering ###

In [44]:
# Population remote salary

# National average salary based on your assumption
national_avg_salary = 104920

# Map experience level to salary multipliers
experience_multiplier = {
    'Entry-level': 0.80,  # 80% of national average
    'Mid-level': 1.00,    # 100% of national average
    'Senior-level': 1.20  # 120% of national average
}

# Adjust national salary based on experience level
df['adjusted_national_salary'] = df['Experience'].map(experience_multiplier) * national_avg_salary

# Fill in Salary_remote with estimated value if it's NaN
df['Salary_remote'] = df.apply(
    lambda row: row['Salary_remote'] if pd.notna(row['Salary_remote']) else row['adjusted_national_salary'] * row['Cost of Living Index']/100,
    axis=1
)

# Show the updated dataframe
df.head()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,Annual Commute Cost,Cost of Living Index,Electricity Price (cents per kWh),adjusted_national_salary
1,IT project manager,Assistant Project Manager,Yogurtland,Farmers Branch,Texas,Entry-level,78500.0,75962.080,31.26,81.66,6119.97,90.5,15.32,83936.0
2,IT project manager,Assistant Program Manager,Azimuth Technology,Naples,Florida,Entry-level,43500.0,75038.784,34.28,83.25,5147.83,89.4,14.20,83936.0
3,IT project manager,Junior Project Manager (IT Aviation),IDMR Solutions Inc,Cleveland,Ohio,Entry-level,44500.0,72017.088,20.79,81.66,3936.90,85.8,15.98,83936.0
4,IT project manager,JUNIOR PROJECT MANAGER/PROJECT ANALYST M/F/D,F. LIST FLORIDA LLC,Opa-locka,Florida,Entry-level,72500.0,73276.128,32.93,83.25,5147.83,87.3,14.20,83936.0
5,IT project manager,"Junior Project Manager in Orlando, FL & San Di...",Exponents Insta USA Inc,San Diego,California,Entry-level,55000.0,88468.544,41.60,81.66,6210.03,105.4,30.55,83936.0


In [45]:
df.shape

(801, 14)

In [46]:
# Calculate remote work expenses: Rent, Electricity, Internet cost, and Annual Commute Cost
df['Remote Work Expenses'] = (
    df['Electricity Price (cents per kWh)'] * 900 * 12 / 100 +  # Annual electricity cost, assuming 900 kWh/month
    df['Avg internet monthly payment'] * 12  # Annual internet cost
)

In [47]:
# Assuming average office space per employee is 150 sqft
average_office_space_per_employee = 150  # square feet

# Calculate office rent for in-office workers
df['Office Rent'] = df['Avg.OfficeRent_per_sqft'] * average_office_space_per_employee * 12

# Calculate the total in-office expenses, including office space rent
df['In-Office Expenses'] = (
    df['Annual Commute Cost'] +  # Annual commute cost
    df['Office Rent']  # Office rent cost for the company
)

In [48]:
df['Total In-office salary'] = df['Salary_in_office'] + df['In-Office Expenses'].fillna(0)
df['Total Remote salary'] = df['Salary_remote'] + df['Remote Work Expenses'].fillna(0)

In [49]:
# Compute Remote vs Office Cost Ratio
df['remote_vs_office_cost_ratio'] = df['Total Remote salary'] / df['Total In-office salary']

In [50]:
df.head()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,Annual Commute Cost,Cost of Living Index,Electricity Price (cents per kWh),adjusted_national_salary,Remote Work Expenses,Office Rent,In-Office Expenses,Total In-office salary,Total Remote salary,remote_vs_office_cost_ratio
1,IT project manager,Assistant Project Manager,Yogurtland,Farmers Branch,Texas,Entry-level,78500.0,75962.080,31.26,81.66,6119.97,90.5,15.32,83936.0,2634.48,56268.0,62387.97,140887.97,78596.560,0.557866
2,IT project manager,Assistant Program Manager,Azimuth Technology,Naples,Florida,Entry-level,43500.0,75038.784,34.28,83.25,5147.83,89.4,14.20,83936.0,2532.60,61704.0,66851.83,110351.83,77571.384,0.702946
3,IT project manager,Junior Project Manager (IT Aviation),IDMR Solutions Inc,Cleveland,Ohio,Entry-level,44500.0,72017.088,20.79,81.66,3936.90,85.8,15.98,83936.0,2705.76,37422.0,41358.90,85858.90,74722.848,0.870298
4,IT project manager,JUNIOR PROJECT MANAGER/PROJECT ANALYST M/F/D,F. LIST FLORIDA LLC,Opa-locka,Florida,Entry-level,72500.0,73276.128,32.93,83.25,5147.83,87.3,14.20,83936.0,2532.60,59274.0,64421.83,136921.83,75808.728,0.553664
5,IT project manager,"Junior Project Manager in Orlando, FL & San Di...",Exponents Insta USA Inc,San Diego,California,Entry-level,55000.0,88468.544,41.60,81.66,6210.03,105.4,30.55,83936.0,4279.32,74880.0,81090.03,136090.03,92747.864,0.681518


In [51]:
#Rounding up decimals
df[['Salary_remote','Remote Work Expenses','Office Rent','In-Office Expenses','Total In-office salary','Total Remote salary','remote_vs_office_cost_ratio']] = df[['Salary_remote','Remote Work Expenses','Office Rent','In-Office Expenses','Total In-office salary','Total Remote salary','remote_vs_office_cost_ratio']].round(2)

In [52]:
df.tail()

,Role,Job_title,Company_in_office,City,State,Experience,Salary_in_office,Salary_remote,Avg.OfficeRent_per_sqft,Avg internet monthly payment,Annual Commute Cost,Cost of Living Index,Electricity Price (cents per kWh),adjusted_national_salary,Remote Work Expenses,Office Rent,In-Office Expenses,Total In-office salary,Total Remote salary,remote_vs_office_cost_ratio
1072,IT project manager,PROJECT DIRECTOR,HRA/DEPT OF SOCIAL SERVICES,None,NaN,Senior-level,119500.0,NaN,NaN,81.66,NaN,NaN,NaN,125904.0,NaN,NaN,NaN,119500.00,NaN,NaN
1073,IT project manager,PROJECT DIRECTOR,HRA/DEPT OF SOCIAL SERVICES,None,NaN,Senior-level,119500.0,NaN,NaN,81.66,NaN,NaN,NaN,125904.0,NaN,NaN,NaN,119500.00,NaN,NaN
1074,IT project manager,PROJECT DIRECTOR,HRA/DEPT OF SOCIAL SERVICES,None,NaN,Senior-level,119500.0,NaN,NaN,81.66,NaN,NaN,NaN,125904.0,NaN,NaN,NaN,119500.00,NaN,NaN
1075,IT project manager,"PROJECT DIRECTOR, IMAGING & ENTERPRISE DATA PR...",HRA/DEPT OF SOCIAL SERVICES,Brooklyn,New York,Senior-level,119500.0,NaN,44.85,81.66,8929.65,NaN,24.37,125904.0,3611.88,80730.0,89659.65,209159.65,NaN,NaN
1077,IT project manager,WDLL Pathway Director (IT),University of the District of Columbia,Washington,DC,Senior-level,77000.0,NaN,52.54,81.66,11067.07,NaN,16.50,125904.0,2761.92,94572.0,105639.07,182639.07,NaN,NaN


In [53]:
df.isna().sum()

Role                                   0
Job_title                              0
Company_in_office                      0
City                                 225
State                                226
Experience                             0
Salary_in_office                       0
Salary_remote                        330
Avg.OfficeRent_per_sqft              246
Avg internet monthly payment           0
Annual Commute Cost                  255
Cost of Living Index                 441
Electricity Price (cents per kWh)    226
adjusted_national_salary               0
Remote Work Expenses                 226
Office Rent                          246
In-Office Expenses                   259
Total In-office salary                 0
Total Remote salary                  330
remote_vs_office_cost_ratio          330
dtype: int64

In [54]:
df = df[~df['State'].isna()]

In [55]:
df.isna().sum()

Role                                   0
Job_title                              0
Company_in_office                      0
City                                   0
State                                  0
Experience                             0
Salary_in_office                       0
Salary_remote                        221
Avg.OfficeRent_per_sqft               20
Avg internet monthly payment           0
Annual Commute Cost                   29
Cost of Living Index                 296
Electricity Price (cents per kWh)      0
adjusted_national_salary               0
Remote Work Expenses                   0
Office Rent                           20
In-Office Expenses                    33
Total In-office salary                 0
Total Remote salary                  221
remote_vs_office_cost_ratio          221
dtype: int64

In [56]:
df.to_csv('Final_Dataset_IT_Project_Manager.csv')

In [57]:
df.shape

(575, 20)

In [58]:
df['City'].nunique()

252